# 过滤低质量基因组数据

前期已经获得216408条数据，现过滤掉低质量基因组数据

In [10]:
from pathlib import Path

tables1_path = Path("../data/TableS1.csv")
checkm_path = Path("../data/checkm.csv")

In [11]:
import pandas as pd

checkm_df = pd.read_csv(checkm_path)
checkm_df = checkm_df.iloc[:, 1:]
checkm_df.head()

,Assembly Accession,CheckM completeness,CheckM contamination
0,GCF_000022305.1,100.0,0.67
1,GCF_000020225.1,100.0,NaN
2,GCF_000025965.1,100.0,0.37
3,GCF_000023105.1,100.0,0.84
4,GCF_000019925.1,100.0,NaN


In [12]:
# 筛选完整度≥90%污染率≤5%的基因组
filtered_df = checkm_df[(checkm_df['CheckM completeness'] >= 90) & (checkm_df['CheckM contamination'] <= 5)]
filtered_df.shape

(188497, 3)

In [13]:
# 读取TableS1
tables1_df = pd.read_csv(tables1_path)
tables1_df.head()

,assembly_accession,16S_ID,assembly_level,Sum,PKSI,PKSother,NRPS,RiPPs,Saccharides,Terpene,PKS-NRP_Hybrids,Others
0,GCF_018967645.1,refseq1,Scaffold,2,0,1,0,0,0,0,0,1
1,GCF_001876165.1,refseq1,Scaffold,5,1,0,0,1,0,1,1,1
2,GCF_001876245.1,refseq1,Scaffold,5,1,0,0,1,0,1,1,1
3,GCF_009600735.1,refseq1,Contig,3,0,0,1,2,0,0,0,0
4,GCF_009600765.1,refseq1,Contig,3,0,0,1,2,0,0,0,0


In [14]:
# 将filtered_df的"Assembly Accession"改为"assembly_accession"
filtered_df = filtered_df.rename(columns={"Assembly Accession": "assembly_accession"})
filtered_df.columns

Index(['assembly_accession', 'CheckM completeness', 'CheckM contamination'], dtype='object')

In [15]:
# 提取合并高质量基因组的16S序列和BGC信息
hq_df = pd.merge(left=filtered_df, left_on=["assembly_accession"], right=tables1_df, right_on=["assembly_accession"], how="inner")
hq_df.shape

(188497, 14)

In [16]:
hq_df.head()

,assembly_accession,CheckM completeness,CheckM contamination,16S_ID,assembly_level,Sum,PKSI,PKSother,NRPS,RiPPs,Saccharides,Terpene,PKS-NRP_Hybrids,Others
0,GCF_000022305.1,100.0,0.67,refseq4905,Complete Genome,3,0,0,0,0,0,1,0,2
1,GCF_000025965.1,100.0,0.37,refseq15376,Complete Genome,7,0,0,0,0,0,1,0,6
2,GCF_000023105.1,100.0,0.84,refseq45559,Complete Genome,1,0,0,0,0,0,1,0,0
3,GCF_000012365.1,100.0,0.06,refseq128457,Complete Genome,20,1,0,5,3,0,2,2,7
4,GCF_000024165.1,100.0,0.88,refseq35003,Complete Genome,8,0,0,2,1,0,1,0,4


In [17]:
hq_df.to_csv("../data/TableS1_HQ.csv", index=False)